### OpenAI Function Calling

The newer OpenAI Function Calling Functionality allows to to define functions which will be passed to the LLM. The LLM
will identify the correct function for the request and provide parameters for the function call.

In [1]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
import openai

def chat(query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
    )
    message = response["choices"][0]["message"]
    return message

In [4]:
query = "How much does pizza salami cost?"
message = chat(query)
message

<OpenAIObject at 0x1bff5e00b30> JSON: {
  "role": "assistant",
  "content": "I am sorry, but as an AI language model, I do not have real-time pricing information. The cost of pizza salami can vary depending on various factors such as location, size of the pizza, quality of ingredients, and the restaurant or pizzeria you are ordering from. It is best to check with local pizzerias or online food delivery services for accurate pricing in your area."
}

To make use of Function calling you need:

1. A function
2. A dictionary which describes the function

In [5]:
import json

def get_pizza_info(pizza_name: str):
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return json.dumps(pizza_info)

In [8]:
functions = [
    {
        "name": "get_pizza_info",
        "description": "Get name and price of a pizza of the restaurant",
        "parameters": {
            "type": "object",
            "properties": {
                "pizza_name": {
                    "type": "string",
                    "description": "The name of the pizza, e.g. Salami",
                },
            },
            "required": ["pizza_name"],
        },
    }
]

In [9]:
def chat(query):
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": query}],
        functions=functions, # this is new
    )
    message = response["choices"][0]["message"]
    return message

In [10]:
chat("What is the capital of france?")

<OpenAIObject at 0x1bff61bd310> JSON: {
  "role": "assistant",
  "content": "The capital of France is Paris."
}

In [11]:
query = "How much does pizza salami cost?"
message = chat(query)
message

<OpenAIObject at 0x1bff61bd970> JSON: {
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_pizza_info",
    "arguments": "{\n\"pizza_name\": \"salami\"\n}"
  }
}

In [12]:
if message.get("function_call"):
    pizza_name = json.loads(message["function_call"]["arguments"]).get("pizza_name")
    print(pizza_name)
    function_response = get_pizza_info(
        pizza_name=pizza_name
    )
    print(function_response)

salami
{"name": "salami", "price": "10.99"}


In [13]:
second_response = openai.ChatCompletion.create(
    model="gpt-3.5-turbo-0613",
    messages=[
        {"role": "user", "content": query},
        message,
        {
            "role": "function",
            "name": "get_pizza_info",
            "content": function_response,
        },
    ],
)
second_response

<OpenAIObject chat.completion id=chatcmpl-8CkPszPIM9bLlWPTduWqq2sMG7Twb at 0x1bfdedc3ef0> JSON: {
  "id": "chatcmpl-8CkPszPIM9bLlWPTduWqq2sMG7Twb",
  "object": "chat.completion",
  "created": 1698049052,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The cost of a pizza salami is $10.99."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 58,
    "completion_tokens": 13,
    "total_tokens": 71
  }
}

The same can be achieved with LangChain

In [14]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains.openai_functions import create_openai_fn_chain

In [15]:

llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)

template = """You are an AI chatbot having a conversation with a human.

Human: {human_input}
AI: """
prompt = PromptTemplate(input_variables=["human_input"], template=template)

chain = create_openai_fn_chain(functions, llm, prompt, verbose=True)

In [16]:
chain.run("How much does pizza salami cost?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation with a human.

Human: How much does pizza salami cost?
AI: 

> Finished chain.


{'pizza_name': 'Salami'}

We can also use Pydantic Classes instead of JSON Schemas

In [17]:
from pydantic import BaseModel, Field

class GetPizzaInfo(BaseModel):
    """Get name and price of a pizza of the restaurant."""

    pizza_name: str = Field(..., description="The name of the pizza, e.g. Salami")

pydantic_classes = [GetPizzaInfo]

In [18]:
chain = create_openai_fn_chain(pydantic_classes, llm, prompt, verbose=True)

In [19]:
chain.run("How much does pizza salami cost?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation with a human.

Human: How much does pizza salami cost?
AI: 

> Finished chain.


{'pizza_name': 'salami'}

We can also pass Functions directly.
To pass Python function in directly, we'll want to make sure our parameters have type hints, we have a docstring, and we use Google Python style docstrings to describe the parameters.

In [20]:
def get_pizza_info(pizza_name: str) -> dict[str, str]:
    """
    Get name and price of a pizza of the restaurant.

    Args:
        pizza_name: The name of the pizza, e.g. Salami.

    Returns:
        Dict[str, str]: A dictionary containing the name and price of the pizza.
    """
    pizza_info = {
        "name": pizza_name,
        "price": "10.99",
    }
    return pizza_info

In [21]:
chain = create_openai_fn_chain([get_pizza_info], llm, prompt, verbose=True)

In [22]:
chain.run("How much does pizza salami cost?")



> Entering new LLMChain chain...
Prompt after formatting:
You are an AI chatbot having a conversation with a human.

Human: How much does pizza salami cost?
AI: 

> Finished chain.


{'pizza_name': 'Salami'}